# Loading images from the Quick Draw Dataset 2k examples


get the data at https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/numpy_bitmap?pli=1


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
import keras.backend as K

import tensorflow as tf

import os
import pickle
import numpy as np

Using TensorFlow backend.


KeyboardInterrupt: 

In [ ]:
print(keras.__version__)
print(tf.__version__)

In [ ]:
num_examples_per_class = 2000

classes = ['airplane','alarm clock','ambulance','angel','ant','anvil','apple','axe','banana','bandage','barn','baseball bat','baseball',
           'basket','basketball','bathtub','beach','bear','beard','bed','bee','belt','bicycle','binoculars','birthday cake','blueberry',
           'book','boomerang','bottlecap','bowtie','bracelet','brain','bread','broom','bulldozer','bus','bus','butterfly','cactus','cake']

In [ ]:
len(classes)

## Load the image data from scratch

Only use if you are downloading the raw data and doing it yourself

## Use this cell to load the premade datasets that I made

In [ ]:
x_data = np.load("./x_data_40_classes_2k.npy")

In [ ]:
x_data.shape

## Lets make some labels for our data

In [ ]:
labels = [np.full((num_examples_per_class,), classes.index(qdraw)) for qdraw in classes]

## Concat the arrays together
y_data = np.concatenate(labels,axis=0)
y_data.shape

In [ ]:
labels

## lets look at the Images 

In [ ]:
def show_object(obj):
    # Reshape 784 array into 28x28 image
    image = obj.reshape([28,28])
    fig, axes = plt.subplots(1, )
    fig.subplots_adjust(hspace=0.5, wspace=0.5)
    plt.imshow(image, cmap='gray_r')
    plt.show()

In [ ]:
show_object(x_data[5300])
print(y_data[5300])

In [ ]:
print(x_data.shape)
print(y_data.shape)

In [ ]:
#shuffling function
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
x_data,y_data = unison_shuffled_copies(x_data,y_data)

In [ ]:
y_data[0]

In [ ]:
show_object(x_data[0])
np.mean(x_data[0])
x_all = np.divide(x_data,255)
y_all = keras.utils.to_categorical(y_data,40)

dataOrder =  np.random.permutation(x_all.shape[0])
trainIdx = dataOrder[0:np.floor(0.8*dataOrder.shape[0]).astype('int32')]
testIdx = dataOrder[(np.floor(0.8*dataOrder.shape[0]).astype('int32')+1):-1]

x_train = x_all[trainIdx]
y_train = y_all[trainIdx]
x_test = x_all[testIdx]
y_test = y_all[testIdx]

In [ ]:
from keras.layers import Reshape, BatchNormalization
from keras.callbacks import ReduceLROnPlateau

def conv2DBNDrRelu(filterSize,kernelSize,dropRate,inputTensor):
    x = Conv2D(filters=filterSize,kernel_size=kernelSize,padding='same')(inputTensor)
    x = BatchNormalization()(x)
    x = Dropout(dropRate)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2,2))(x)
    return x

Inp = Input(shape=(784,),dtype='float32')
x = BatchNormalization()(Inp)
x = Reshape((28,28,1), input_shape=(784,))(Inp)
x = conv2DBNDrRelu(64,(3,3),0.2,x)
x = conv2DBNDrRelu(64,(3,3),0.2,x)
x = conv2DBNDrRelu(128,(3,3),0.2,x)
x = Flatten()(x)
x = Dense(512, activation='relu', name = "Dense_1")(x)
x = Dropout(0.4)(x)
x = Dense(512, activation='relu', name = "Dense_2")(x)
output = Dense(40, activation='softmax', name = "Outputlayer")(x)

model = Model(Inp, output)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=2, min_lr=0.00001)
#learning_rate = 0.1
#K.set_value(model.optimizer.lr, learning_rate)

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=40,
                    verbose=1, # This is for what we want it to display out as it trains 
                    validation_data=(x_test, y_test))

In [ ]:
model.save('./quickDrawModel.h5')

In [ ]:
allTestResults = model.predict(x_test[np.newaxis,0])
print(allTestResults)
error = 1 - allTestResults[0][np.argmax(allTestResults)]
print(error)

In [43]:
allTestResults = model.predict(x_test)
topChoices = np.argmax(allTestResults,axis=-1)

In [62]:
print(allTestResults.shape)
print(topChoices.shape)

error =  allTestResults[:,topChoices]
print(error.shape)

(15998, 40)
(15998,)
(15998, 15998)


In [ ]:
model.predict(x_test,32)